In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import random

from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
X_train = torch.load('/kaggle/input/pulsedb-dataset-pytorch-tensors/X_train.pt')
Y_train = torch.load('/kaggle/input/pulsedb-dataset-pytorch-tensors/Y_train.pt')
X_test = torch.load('/kaggle/input/pulsedb-dataset-pytorch-tensors/X_test.pt')
Y_test = torch.load('/kaggle/input/pulsedb-dataset-pytorch-tensors/Y_test.pt')

X_train[:, :1250] = 2 * X_train[:, :1250] - 1
X_test[:, :1250] = 2 * X_test[:, :1250] - 1

## Dataset

In [ ]:
# Dataset and DataLoader
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

In [ ]:
# Hyperparameters
input_size = 1
hidden_size = 64
output_size = 1
num_layers = 2
batch_size = 64
num_epochs = 100
learning_rate = 0.001

# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)

# Multi-GPU support
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

## RNN

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # Initialize hidden state
        out, _ = self.rnn(x, h0)  # Pass input through RNN layers
        out = self.fc(out)  # Fully connected layer for output
        return out

In [ ]:
# # Hyperparameters
# input_size = 1
# hidden_size = 64
# output_size = 1
# num_layers = 2
# batch_size = 64
# num_epochs = 100
# learning_rate = 0.001

# # Model, Loss, Optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = RNNModel(input_size, hidden_size, output_size, num_layers).to(device)

# # Multi-GPU support
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

## Transformer

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead=8, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, hidden_size*4, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(src.shape[2])
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
# # Hyperparameters
# input_size = 1
# hidden_size = 64
# output_size = 1
# num_layers = 4
# batch_size = 64
# num_epochs = 100
# learning_rate = 0.001

# # Model, Loss, Optimizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = TransformerModel(input_size, hidden_size, output_size, num_layers).to(device)

# # Multi-GPU support
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs")
#     model = nn.DataParallel(model)

## Train

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def plot_random_sample(model, X_test, Y_test, device, epoch, batch=None):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        random_idx = random.randint(0, len(X_test) - 1)  # Random sample index
        sample_input = X_test[random_idx, :1250].unsqueeze(0).unsqueeze(2).to(device)  # Shape [1, 1250, 1]
        sample_output = model(sample_input)  # Get the model's prediction

        # Plot actual vs predicted ABP
        plt.figure(figsize=(10, 5))
        plt.plot(Y_test[random_idx].cpu().numpy(), label='Actual', color='red')
        plt.plot(sample_output.cpu().squeeze().detach().numpy(), label='Predicted', color='blue')
        if batch:
            plt.title(f'Epoch {epoch + 1} | Batch {batch+1}: Random Sample Prediction')
        else:
            plt.title(f'Epoch {epoch + 1}: Random Sample Prediction')
        plt.xlabel('Time')
        plt.ylabel('ABP')
        plt.legend()
        plt.show()
    model.train()

In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data = data[:, :1250].unsqueeze(2).to(device)  # Shape: [batch, 1250, 1]
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        # Plot random sample every 10 epochs
        if True:
            plot_random_sample(model, X_test, Y_test, device, epoch, batch_idx)

    # Validation
    model.eval() 
    val_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data[:, :1250].unsqueeze(2).to(device)
            target = target.to(device)
            output = model(data)
            val_loss += criterion(output.squeeze(), target).item()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(test_loader):.4f}')
    

print("Training completed.")